In [ ]:
import os
from google.colab import drive
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import np_utils
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
from pathlib import Path
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

drive.mount('/content/drive')

train_data_dir = '/content/drive/MyDrive/딥러닝/Colab Notebook/train'
categories = ['adidas', 'converse', 'nike']
num_classes = len(categories)

#이미지 크기
image_w = 240
image_h = 240
pixels = image_h*image_w*3

#trainset원핫코딩
X1 = []
y1 = []
for idx, category in enumerate(categories):
  label = [0 for i in range(num_classes)]
  label[idx] = 1

  train_image_dir = train_data_dir + '/' + category
  files = glob.glob(train_image_dir + '/*.jpg')
  print(category, "파일 길이:", len(files))

  for i,f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)

    X1.append(data)
    y1.append(label)
  
X1 = np.array(X1)
y1 = np.array(y1)

test_data_dir = '/content/drive/MyDrive/딥러닝/Colab Notebook/test'

#testset원핫코딩
X2 = []
y2 = []
for idx, category in enumerate(categories):
  label = [0 for i in range(num_classes)]
  label[idx] = 1

  test_image_dir = test_data_dir + '/' + category
  files = glob.glob(test_image_dir + '/*.jpg')
  print(category, "파일 길이:", len(files))

  for i,f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)

    X2.append(data)
    y2.append(label)
  
X2 = np.array(X2)
y2 = np.array(y2)

X_train = X1
y_train = y1
X_test = X2
y_test = y2

X_train = X_train.astype('float64')
X_train = X_train/255
X_test = X_test.astype('float64')
X_test = X_test/255

model = Sequential()
model.add(Conv2D(16, kernel_size = (3,3), input_shape = (240, 240, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.25))
model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.25))
model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(3, activation = 'softmax'))
model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modelpath = '/content/drive/MyDrive/딥러닝/모델저장용'
checkpointer = ModelCheckpoint(filepath = modelpath, monitor = 'val_loss', verbose = 1, save_best_only = True)
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10)
history = model.fit(X_train, y_train, validation_split = 0.25, epochs = 1000, batch_size = 100, verbose = 0, callbacks = [early_stopping_callback, checkpointer])
print("\n Test accuracy: {}.".format(model.evaluate(X_test)[1]))

pred = model.predict(X_test)
pred = np.argmax(pred,axis=1)
cf_matrix = confusion_matrix(y_test, pred, normalize='true')
plt.figure(figsize = (10,7))
sns.heatmap(cf_matrix, annot=False, xticklabels = sorted(set(y_test)), yticklabels = sorted(set(y_test)),cbar=False)
plt.title('Normalized Confusion Matrix', fontsize = 23)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.show()


'''
train_adidas_data = Path('/content/drive/MyDrive/딥러닝/Colab Notebook/train/adidas')
filepaths1 = list(train_adidas_data.glob(r'**/*.jpg'))
train_converse_data = Path('/content/drive/MyDrive/딥러닝/Colab Notebook/train/converse')
filepaths2 = list(train_converse_data.glob(r'**/*.jpg'))
train_nike_data = Path('/content/drive/MyDrive/딥러닝/Colab Notebook/train/nike')
filepaths3 = list(train_nike_data.glob(r'**/*.jpg'))

test_adidas_data = Path('/content/drive/MyDrive/딥러닝/Colab Notebook/test/adidas')
filepaths4 = list(test_adidas_data.glob(r'**/*.jpg'))
test_converse_data = Path('/content/drive/MyDrive/딥러닝/Colab Notebook/test/converse')
filepaths5 = list(test_converse_data.glob(r'**/*.jpg'))
test_nike_data = Path('/content/drive/MyDrive/딥러닝/Colab Notebook/test/nike')
filepaths6 = list(test_nike_data.glob(r'**/*.jpg'))

def proc_img(filepath):
    """
   		이미지데이터의 경로와 label데이터로 데이터프레임 만들기 
    """

    labels = [str(filepath[i]).split("/")[-2] \
              for i in range(len(filepath))]

    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    # 경로와 라벨 concatenate
    df = pd.concat([filepath, labels], axis=1)

    # index 재설정
    df = df.sample(frac=1,random_state=0).reset_index(drop = True)
    
    return df

train_df_adidas = proc_img(filepaths1)
train_df_converse = proc_img(filepaths2)
train_df_nike = proc_img(filepaths3)

test_df_adidas = proc_img(filepaths4)
test_df_converse = proc_img(filepaths5)
test_df_nike = proc_img(filepaths6)

train_df = pd.concat([train_df_adidas,train_df_converse,train_df_nike], ignore_index=True)
test_df = pd.concat([test_df_adidas,test_df_converse,test_df_nike], ignore_index=True)


train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split=0.2)

train_gen = train_datagen.flow_from_directory('/content/drive/MyDrive/딥러닝/Colab Notebook/train',
                                                 target_size = (150, 150),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',subset='training')
val_gen  = train_datagen.flow_from_directory('/content/drive/MyDrive/딥러닝/Colab Notebook/train',
                                                 target_size = (150, 150),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',subset='validation')
# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[150, 150, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=8, activation='softmax'))

# Compiling the CNN
cnn.compile(optimizer = 'adam', 
            loss = 'categorical_crossentropy', 
            metrics = ['accuracy'])
cnn.summary()

cnn.fit(x = train_gen, validation_data = val_gen, epochs = 10)
'''

# 새 섹션